# Нейронная сеть для считывания знаков STOP с рисунка

## Подключение библиотек

In [7]:
import os
import json
import random
import shutil
from ultralytics import YOLO

## Работа с данными

### Преобразоавние .json меток в .txt в формате, удобном для yolo

In [8]:
images_dir = "./dataset/ds/img"
annotations_dir = "./dataset/ds/ann"
output_dir = "./dataset/ds/yolo_labels"
meta_file = "./dataset/meta.json"

os.makedirs(output_dir, exist_ok=True)

with open(meta_file, "r", encoding="utf-8") as f:
    meta_data = json.load(f)

classes = { cls["title"].lower(): idx for idx, cls in enumerate(meta_data["classes"]) }

if "stop" not in classes:
    raise ValueError("Класс 'stop' не найден в meta.json!")

def normalize_bbox(bbox, img_width, img_height):
    x_min, y_min = bbox[0]
    x_max, y_max = bbox[1]
    x_center = (x_min + x_max) / 2 / img_width
    y_center = (y_min + y_max) / 2 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return x_center, y_center, width, height

for annotation_file in os.listdir(annotations_dir):
    if annotation_file.endswith(".json"):
        with open(os.path.join(annotations_dir, annotation_file), "r", encoding="utf-8") as f:
            data = json.load(f)
        
        img_width = data["size"]["width"]
        img_height = data["size"]["height"]
        
        yolo_labels = []
        for obj in data["objects"]:
            class_title = obj["classTitle"].lower()
            if class_title == "stop": 
                bbox = obj["points"]["exterior"]
                x_center, y_center, width, height = normalize_bbox(bbox, img_width, img_height)
                yolo_labels.append(f"0 {x_center} {y_center} {width} {height}")
        
        image_name = os.path.splitext(annotation_file)[0].replace('.png', '.txt')
        label_file = os.path.join(output_dir, image_name)
        
        with open(label_file, "w") as f:
            if yolo_labels: 
                f.write("\n".join(yolo_labels))

### Разделение данных на тренировачные, валидационные и тестируемые

In [9]:
images_dir = "./dataset/ds/img"
labels_dir = "./dataset/ds/yolo_labels"
dataset_dir = "./data"
train_images_dir = os.path.join(dataset_dir, "train", "images")
train_labels_dir = os.path.join(dataset_dir, "train", "labels")
val_images_dir = os.path.join(dataset_dir, "valid", "images")
val_labels_dir = os.path.join(dataset_dir, "valid", "labels")
test_images_dir = os.path.join(dataset_dir, "test", "images")
test_labels_dir = os.path.join(dataset_dir, "test", "labels")

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)

image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]

random.shuffle(image_files)

test_size = 0.2  # 20% для теста
val_size = 0.1   # 10% для валидации
split_index_test = int(len(image_files) * (1 - test_size))
split_index_val = int(len(image_files) * (1 - test_size - val_size))

train_files = image_files[:split_index_val]
val_files = image_files[split_index_val:split_index_test]
test_files = image_files[split_index_test:]

for file in train_files:
    shutil.copy(os.path.join(images_dir, file), os.path.join(train_images_dir, file))
    shutil.copy(os.path.join(labels_dir, file.replace('.png', '.txt')), os.path.join(train_labels_dir, file.replace('.png', '.txt')))

for file in val_files:
    shutil.copy(os.path.join(images_dir, file), os.path.join(val_images_dir, file))
    shutil.copy(os.path.join(labels_dir, file.replace('.png', '.txt')), os.path.join(val_labels_dir, file.replace('.png', '.txt')))

for file in test_files:
    shutil.copy(os.path.join(images_dir, file), os.path.join(test_images_dir, file))
    shutil.copy(os.path.join(labels_dir, file.replace('.png', '.txt')), os.path.join(test_labels_dir, file.replace('.png', '.txt')))
    
shutil.rmtree("./dataset/ds/yolo_labels")

## Тренировка

In [ ]:
!yolo detect train data=data.yaml model=yolov8n.pt epochs=50 imgsz=640

## Тестирование

In [ ]:
model = YOLO("runs/detect/train5/weights/best.pt")  
metrics = model.val(data="data.yaml", split="test")
print(f"Quality functionality mAP50: { round(metrics.box.map50 * 100, 5) }%")  # Основная метрика

Ultralytics 8.3.103  Python-3.11.0 torch-2.0.1+cpu CPU (AMD Ryzen 7 7730U with Radeon Graphics)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Projects\PetProjects\StopSignDetector\dataset\test\labels.cache... 176 images, 157 backgrounds, 0 corrupt: 100%|██████████| 176/176 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:15<00:00,  1.45s/it]


                   all        176         19      0.944          1      0.993      0.875
Speed: 1.6ms preprocess, 82.3ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to C:\Projects\PetProjects\StopSignDetector\runs\detect\val4
Quality functionality mAP50: 99.25%
